<a href="https://colab.research.google.com/github/navneetkrc/deployment_ready_ml_models/blob/master/keras_toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BentoML Example: Keras Toxic Comment Classification


[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to turn a Keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.


This notebook is built based on: https://www.kaggle.com/sarvajna/keras-sequential-model-lb-0-052

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=keras&ea=keras-toxic-comment-classification&dt=keras-toxic-comment-classification)

Source  https://github.com/bentoml/BentoML

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
!pip install bentoml
!pip install keras kaggle tensorflow==1.14.0

In [0]:
import bentoml
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split

In [0]:
list_of_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
max_features = 20000
max_text_length = 400
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
batch_size = 32
epochs = 2

## Prepare Dataset

Please Download data with Kaggle at https://www.kaggle.com/sarvajna/keras-sequential-model-lb-0-052/data

If you are running this notebook in Google Colab, fill in your kaggle credential below and download the training dataset from Kaggle:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/MLAI_Datasets/jigsaw-toxic-comment-classification-challenge.zip'

Archive:  /content/drive/My Drive/MLAI_Datasets/jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [0]:
!unzip train.csv.zip
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  test_labels.csv.zip
  inflating: test_labels.csv         


In [0]:
train_df = pd.read_csv('./train.csv')

print(train_df.head())

                 id  ... identity_hate
0  0000997932d777bf  ...             0
1  000103f0d9cfb60f  ...             0
2  000113f07ec002fd  ...             0
3  0001b41b1c6bb37e  ...             0
4  0001d958c54c6e35  ...             0

[5 rows x 8 columns]


In [0]:
x = train_df['comment_text'].values
print(x)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [0]:
y = train_df[list_of_classes].values
print(y)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [0]:
x_tokenizer = text.Tokenizer(num_words=max_features)
print(x_tokenizer)
x_tokenizer.fit_on_texts(list(x))
print(x_tokenizer)
x_tokenized = x_tokenizer.texts_to_sequences(x) #list of lists(containing numbers), so basically a list of sequences, not a numpy array
#pad_sequences:transform a list of num_samples sequences (lists of scalars) into a 2D Numpy array of shape 
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y, test_size=0.1, random_state=1)

In [0]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=max_text_length))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto 6 output layers, and squash it with a sigmoid:
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Build model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
______________________

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
validation_data=(x_val, y_val))

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 564s 4ms/step - loss: 0.0642 - acc: 0.9785 - val_loss: 0.0533 - val_acc: 0.9806
Epoch 2/2
143613/143613 [==============================] - 563s 4ms/step - loss: 0.0464 - acc: 0.9826 - val_loss: 0.0496 - val_acc: 0.9814


In [0]:
test_df = pd.read_csv('./test.csv')

In [0]:
x_test = test_df['comment_text'].values

In [0]:
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
x_testing = sequence.pad_sequences(x_test_tokenized, maxlen=max_text_length)

In [0]:
y_testing = model.predict(x_testing, verbose = 1)

153164/153164 [==============================] - 132s 861us/step


In [0]:
sample_submission = pd.read_csv("./sample_submission.csv")
sample_submission[list_of_classes] = y_testing
sample_submission.to_csv("toxic_comment_classification.csv", index=False)

## Create BentoService for model serving

In [0]:
%%writefile toxic_comment_classifier.py

from bentoml import api, artifacts, env, BentoService
from bentoml.artifact import PickleArtifact, KerasModelArtifact
from bentoml.handlers import DataframeHandler

from keras.preprocessing import text, sequence
import numpy as np

list_of_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
max_text_length = 400

@env(pip_dependencies=['tensorflow==1.14.0', 'keras', 'pandas', 'numpy'])
@artifacts([PickleArtifact('x_tokenizer'), KerasModelArtifact('model')])
class ToxicCommentClassification(BentoService):
    
    def tokenize_df(self, df):
        comments = df['comment_text'].values
        tokenized = self.artifacts.x_tokenizer.texts_to_sequences(comments)        
        input_data = sequence.pad_sequences(tokenized, maxlen=max_text_length)
        return input_data
    
    @api(DataframeHandler)
    def predict(self, df):
        input_data = self.tokenize_df(df)
        prediction = self.artifacts.model.predict(input_data)
        result = []
        for i in prediction:
            result.append(list_of_classes[np.argmax(i)])
        return result

Overwriting toxic_comment_classifier.py


## Save BentoService to file archive

In [0]:
# 1) import the custom BentoService defined above
from toxic_comment_classifier import ToxicCommentClassification

# 2) `pack` it with required artifacts
svc = ToxicCommentClassification()
svc.pack('x_tokenizer', x_tokenizer)
svc.pack('model', model)

# 3) save your BentoSerivce
saved_path = svc.save()

## Load BentoService from archive

In [0]:
sample_test = test_df.iloc[40:42]
print(sample_test)
bento_service = bentoml.load(saved_path)

print(bento_service.predict(sample_test))

                  id                                       comment_text
40  0011cefc680993ba                      REDIRECT Talk:Mi Vida Eres Tú
41  0011ef6aa33d42e6  " \n I'm not convinced that he was blind. Wher...
['toxic', 'toxic']


In [0]:
#Need to store these services name such that we can easily use them for the cells that follow
!bentoml get ToxicCommentClassification

BENTO_SERVICE                                     AGE                           APIS                       ARTIFACTS
ToxicCommentClassification:20200214093630_7EDBFF  44 minutes and 21.78 seconds  predict<DataframeHandler>  x_tokenizer<PickleArtifact>, model<KerasModelArtifact>
ToxicCommentClassification:20200214082616_B70B48  1 hour and 54 minutes         predict<DataframeHandler>  x_tokenizer<PickleArtifact>, model<KerasModelArtifact>


In [0]:
!bentoml get ToxicCommentClassification:20200214093630_7EDBFF 

{
  "name": "ToxicCommentClassification",
  "version": "20200214093630_7EDBFF",
  "uri": {
    "type": "LOCAL",
    "uri": "/root/bentoml/repository/ToxicCommentClassification/20200214093630_7EDBFF"
  },
  "bentoServiceMetadata": {
    "name": "ToxicCommentClassification",
    "version": "20200214093630_7EDBFF",
    "createdAt": "2020-02-14T09:36:56.615243Z",
    "env": {
      "condaEnv": "name: bentoml-ToxicCommentClassification\nchannels:\n- defaults\ndependencies:\n- python=3.6.9\n- pip\n",
      "pipDependencies": "bentoml==0.6.2\ntensorflow==1.14.0\nkeras\npandas\nnumpy",
      "pythonVersion": "3.6.9"
    },
    "artifacts": [
      {
        "name": "x_tokenizer",
        "artifactType": "PickleArtifact"
      },
      {
        "name": "model",
        "artifactType": "KerasModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "

In [0]:
!bentoml info ToxicCommentClassification:20200214093630_7EDBFF 

{
  "name": "ToxicCommentClassification",
  "version": "20200214093630_7EDBFF",
  "created_at": "2020-02-14T09:36:56.615243Z",
  "env": {
    "conda_env": "name: bentoml-ToxicCommentClassification\nchannels:\n- defaults\ndependencies:\n- python=3.6.9\n- pip\n",
    "pip_dependencies": "bentoml==0.6.2\ntensorflow==1.14.0\nkeras\npandas\nnumpy",
    "python_version": "3.6.9"
  },
  "artifacts": [
    {
      "name": "x_tokenizer",
      "artifact_type": "PickleArtifact"
    },
    {
      "name": "model",
      "artifact_type": "KerasModelArtifact"
    }
  ],
  "apis": [
    {
      "name": "predict",
      "handler_type": "DataframeHandler",
      "docs": "BentoService API",
      "handler_config": {
        "orient": "records",
        "typ": "frame",
        "input_dtypes": null,
        "output_orient": "records"
      }
    }
  ]
}


In [0]:
!bentoml run ToxicCommentClassification:20200213141504_137C94 predict --input '[{"comment_text": "bad terrible"}]'

Traceback (most recent call last):
  File "/usr/local/bin/bentoml", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/bentoml/cli/click_utils.py", line 94, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/bentoml/cli/__init__.py", line 146, in run
    bento, pip_installed_bundle_path
  File "

## Use BentoService as PyPI package

In [0]:
!pip install {saved_path}

Processing /root/bentoml/repository/ToxicCommentClassification/20200214093630_7EDBFF
  Created wheel for ToxicCommentClassification: filename=ToxicCommentClassification-20200214093630_7EDBFF-cp36-none-any.whl size=16002720 sha256=9adb7bab7a011f29a501f413d486cc74bba039082f98dfced5dc1c7ae23300f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-gwziowqr/wheels/01/51/03/74062e09c52acb297c2ed760b536388f4f0bed198c40f59266
Successfully built ToxicCommentClassification
  Found existing installation: ToxicCommentClassification 20200214082616-B70B48
    Uninstalling ToxicCommentClassification-20200214082616-B70B48:
      Successfully uninstalled ToxicCommentClassification-20200214082616-B70B48


In [0]:
import ToxicCommentClassification

svc = ToxicCommentClassification.load()
result = svc.predict(sample_test)
result

['toxic', 'toxic']

## Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS Sagemaker

**Use `bentoml sagemaker deploy` to deploy BentoService to AWS Sagemaker**

In [0]:
!bentoml sagemaker deploy keras-toxic -b ToxicCommentClassification:20200214093630_7EDBFF \
    --api-name predict --verbose

[2020-02-14 10:23:09,161] DEBUG - Using BentoML with local Yatai server
[2020-02-14 10:23:09,285] DEBUG - Upgrading tables to the latest revision
Failed to create AWS Sagemaker deployment keras-toxic.: Deployment "keras-toxic" already existed, use Update or Apply for updating existing deployment, delete the deployment, or use a different deployment name


`bentoml sagemaker list` displays all deployed Sagemaker deployments

In [0]:
!bentoml sagemaker list

NAME         NAMESPACE    PLATFORM       BENTO_SERVICE                                     STATUS    AGE
keras-toxic  dev          aws-sagemaker  ToxicCommentClassification:20200213141504_137C94  pending   1 hour and 55 minutes


`bentoml sagemaker get` retrieve the latest status of Sagemaker deployment

In [0]:
!bentoml sagemaker get keras-toxic

Traceback (most recent call last):
  File "/usr/local/bin/bentoml", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/bentoml/cli/click_utils.py", line 94, in wrapper
    return func(*args, *

Validate and test Sagemaker deployment with sample data

In [0]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-keras-toxic \
--body '[{"comment_text": "bad terrible"}]' --content-type application/json output.json && cat output.json

/bin/bash: aws: command not found


`bentoml sagemaker delete` will remove Sagmaker deployment and related resources

In [0]:
!bentoml sagemaker delete keras-toxic

Traceback (most recent call last):
  File "/usr/local/bin/bentoml", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/bentoml/cli/click_utils.py", line 94, in wrapper
    return func(*args, *